In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import re

#### Import the data and get a high-level picture

In [ ]:
df = pd.read_csv('sales.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

#### TODO: Fix column datatypes

Change ordered_at to datetime

Change price and line_total to float

In [ ]:
df.dtypes

#### TODO: drop if duplicated or null

In [ ]:
df[df.duplicated()].shape[0]

In [ ]:
df.isnull().sum()

In [ ]:
df[df['name'].isnull()].head()

#### Sanity check for value ranges and to check assumptions

In [ ]:
df[(df['price'] * df['quantity']) != df['line_total']].shape[0]

In [ ]:
df[df['line_total'] < 0].shape[0]

#### TODO: 
Set line_total = price * quantity if different
Remove if line total < 0

In [ ]:
df.describe()

#### TODO: Get value between "" in name and put it in category column

In [ ]:
df.head()

#### Analysis, finally!

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
df.groupby('name')['line_total'].sum().sort_values(ascending=False).head(10).plot(kind='bar')
f.autofmt_xdate()
plt.show()